In [1]:
import numpy as np
import math
import scipy.stats as stats
from scipy.optimize import minimize
import pandas as pd
import statsmodels.api as sm

from scipy.stats import norm
from sklearn.decomposition import PCA

In [38]:
portfolio = pd.read_csv("portfolio.csv")
returns = pd.read_csv('DailyReturn.csv')
prices = pd.read_csv('DailyPrices.csv')
portfolio = portfolio[~portfolio['Stock'].isin(['ELV', 'MMC', 'VRTX', 'REGN', 'CB', 'CI', 'ETN', 'SLB', 'PGR', 'BSX'])]
portfolio = portfolio[portfolio['Portfolio'] == 'A']
portfolio


,Portfolio,Stock,Holding
0,A,AAPL,158
1,A,MSFT,178
2,A,AMZN,110
3,A,NVDA,54
4,A,GOOGL,69
5,A,TSLA,60
6,A,GOOG,167
7,A,BRK-B,185
8,A,META,84
9,A,UNH,61


In [28]:
def ewCovar(returns, lambda_=0.97):
    returns = returns.values
    mean_return = np.mean(returns, axis=0)
    normalized_returns = returns - mean_return

    n_timesteps = normalized_returns.shape[0]
    cov = np.cov(returns, rowvar=False)

    for t in range(1, n_timesteps):
        cov = lambda_ * cov + (1 - lambda_) * np.outer(normalized_returns[t], normalized_returns[t])
    return cov

In [37]:
# Multiply each holding by its corresponding latest stock price to get the asset value
asset_values = portfolio["Holding"].values.reshape(-1, 1) * prices[portfolio["Stock"]].tail(1).T.values
# Calculate delta by dividing the asset values by the total current price of the portfolio
delta = asset_values / current_price  
np.shape(delta)
np.shape(assets_cov)

(101, 101)

In [39]:
#calculate returns
returns = returns[portfolio['Stock']]

In [42]:
#current portfolio price = lastest price of each equity combine with holding value 
current_price_port = np.dot(prices[portfolio["Stock"]].tail(1), portfolio["Holding"])

#ew covariance matrix
assets_cov = ewCovar(returns, 0.7)

#delta(portfolio return's derivative relative to each equity's change) calculation

# Multiply each holding by its corresponding latest stock price to get the asset value
asset_values = portfolio["Holding"].values.reshape(-1, 1) * prices[portfolio["Stock"]].tail(1).T.values
# Calculate delta by dividing the asset values by the total current price of the portfolio
delta = asset_values / current_price       

p_sig = np.sqrt(np.transpose(delta) @ assets_cov @ delta)
#formula for delta_normal method
delta_var = -current_price_port * stats.norm.ppf(0.05) * p_sig
delta_var 

array([[18130.1570797]])